<a href="https://colab.research.google.com/github/Matheusvfvitor/R/blob/master/RMachine_Learning_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo :

Realizar a avaliação da base de dados crédito e realizar a predição da liberação ou não de crédito de acordo com os dados recebidos.

Para realização desse estudo realizaremos as seguintes etapas

1. Carregamento dos dados da base dados créditos (*.csv)
2. Limpeza dos valores missing e nulls
3. Normalização e Padronização dos dados
4. Separação da base de dados em treino e teste
5. Criação do classificador na base dados de treino
6. Treino na base
7. Aplicação do Algorítmo de predição SVM
8. Gerar a matriz de confusão com os resultados
9. Aplicação do pacote Caret e para quantificação dos resultados





In [0]:
## Importando Pacotes

library(tidyr)
library(dplyr)
library(stringr)
library(ggplot2)

In [61]:
## 1)
## Fazendo o carregamento do banco de dados
data = read.csv("census.csv",encoding="UTF-8")

head(data)
str(data)

# Apagando a Coluna Id
data$X =NULL

,X,age,workclass,final.weight,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loos,hour.per.week,native.country,income
,<int>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>
1,1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
2,2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
3,3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
4,4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
5,5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
6,6,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


'data.frame':	30162 obs. of  16 variables:
 $ X             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ age           : int  39 50 38 53 28 37 49 52 31 42 ...
 $ workclass     : Factor w/ 7 levels " Federal-gov",..: 6 5 3 3 3 3 3 5 3 3 ...
 $ final.weight  : int  77516 83311 215646 234721 338409 284582 160187 209642 45781 159449 ...
 $ education     : Factor w/ 16 levels " 10th"," 11th",..: 10 10 12 2 10 13 7 12 13 10 ...
 $ education.num : int  13 13 9 7 13 14 5 9 14 13 ...
 $ marital.status: Factor w/ 7 levels " Divorced"," Married-AF-spouse",..: 5 3 1 3 3 3 4 3 5 3 ...
 $ occupation    : Factor w/ 14 levels " Adm-clerical",..: 1 4 6 6 10 4 8 4 10 4 ...
 $ relationship  : Factor w/ 6 levels " Husband"," Not-in-family",..: 2 1 2 1 6 6 2 1 2 1 ...
 $ race          : Factor w/ 5 levels " Amer-Indian-Eskimo",..: 5 5 5 3 3 5 3 5 5 5 ...
 $ sex           : Factor w/ 2 levels " Female"," Male": 2 2 2 2 1 1 1 2 1 2 ...
 $ capital.gain  : int  2174 0 0 0 0 0 0 0 14084 5178 ...
 $ capital.loos  : int  

In [62]:
## 2) Tratando transformando os valores em factors

table(data$sex) ##Visualisando uma tabela específica resumo sobre os valores de sexo

data$sex <- factor(data$sex, levels = c(" Female", " Male"), labels = c(0,1)) # Criando um fator de 0 ou 1 para os sexos masculinos e femininos.
#Workclass

table(data$workclass)
data$workclass <- factor(data$workclass, levels = c(" Federal-gov", " Local-gov", " Private", " Self-emp-inc", " Self-emp-not-inc", " State-gov", " Without-pay"), labels = c(0,1,2,3,4,5,6))
data$education <- factor(data$education, levels = c(" 10th", " 11th", " 12th", " 1st-4th", " 5th-6th", " 7th-8th", " 9th", " Assoc-acdm", " Assoc-voc", " Bachelors", " Doctorate", " HS-grad", " Masters", " Preschool", " Prof-school", " Some-college"), labels = c(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15))
data$marital.status <- factor(data$marital.status, levels = c(" Divorced", " Married-AF-spouse", " Married-civ-spouse", " Married-spouse-absent", " Never-married", " Separated", " Widowed"), labels = c(0,1,2,3,4,5,6))
data$occupation <- factor(data$occupation, levels = c(" Adm-clerical", " Armed-Forces", " Craft-repair"," Exec-managerial"," Farming-fishing", " Handlers-cleaners", " Machine-op-inspct", " Other-service", " Priv-house-serv", " Prof-specialty", " Protective-serv", " Sales", " Tech-support", " Transport-moving"), labels = c(0,1,2,3,4,5,6,7,8,9,10,11,12,13))
u <- unique(data$relationship) # criando um vetor de dados únicos 
as.vector(u)

data$relationship <- factor(data$relationship, levels = u, labels = c(0,1,2,3,4,5))
r <- unique(data$race)
as.vector(r)

data$race <- factor(data$race, levels = r, labels = c(0,1,2,3,4))

nc <- unique(data$native.country)
as.vector(nc)

data$native.country <- factor(data$native.country, levels = nc, labels = c(0:40))
data$income <- factor(data$income, levels = c(" <=50K", " >50K"), labels = c(0,1))

head(data)



 Female    Male 
   9782   20380 


      Federal-gov         Local-gov           Private      Self-emp-inc 
              943              2067             22286              1074 
 Self-emp-not-inc         State-gov       Without-pay 
             2499              1279                14 

[1] " Not-in-family"  " Husband"        " Wife"           " Own-child"     
[5] " Unmarried"      " Other-relative"

[1] " White"              " Black"              " Asian-Pac-Islander"
[4] " Amer-Indian-Eskimo" " Other"

[1] " United-States"              " Cuba"                      
 [3] " Jamaica"                    " India"                     
 [5] " Mexico"                     " Puerto-Rico"               
 [7] " Honduras"                   " England"                   
 [9] " Canada"                     " Germany"                   
[11] " Iran"                       " Philippines"               
[13] " Poland"                     " Columbia"                  
[15] " Cambodia"                   " Thailand"                  
[17] " Ecuador"                    " Laos"                      
[19] " Taiwan"                     " Haiti"                     
[21] " Portugal"                   " Dominican-Republic"        
[23] " El-Salvador"                " France"                    
[25] " Guatemala"                  " Italy"                     
[27] " China"                      " South"                     
[29] " Japan"                      " Yugoslavia"                
[31] " Peru"                       " Outlying-US(Guam-USVI-etc)"
[33] " Scotland"                   " Trinadad&Tobago"           
[35] " Greece"                     " Nicaragua"                 
[37] " Vietnam"                    " Hong"                      
[39] " Ireland"                    " Hungary"                   
[41] " Holand-Netherlands"

,age,workclass,final.weight,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loos,hour.per.week,native.country,income
,<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>
1,39,5,77516,9,13,4,0,0,0,1,2174,0,40,0,0
2,50,4,83311,9,13,2,3,1,0,1,0,0,13,0,0
3,38,2,215646,11,9,0,5,0,0,1,0,0,40,0,0
4,53,2,234721,1,7,2,5,1,1,1,0,0,40,0,0
5,28,2,338409,9,13,2,9,2,1,0,0,0,40,1,0
6,37,2,284582,12,14,2,3,2,0,0,0,0,40,0,0


In [41]:
str(data)
summary(data)
head(data)

'data.frame':	30162 obs. of  15 variables:
 $ age           : int  39 50 38 53 28 37 49 52 31 42 ...
 $ workclass     : Factor w/ 7 levels " Federal-gov",..: 6 5 3 3 3 3 3 5 3 3 ...
 $ final.weight  : int  77516 83311 215646 234721 338409 284582 160187 209642 45781 159449 ...
 $ education     : Factor w/ 16 levels " 10th"," 11th",..: 10 10 12 2 10 13 7 12 13 10 ...
 $ education.num : int  13 13 9 7 13 14 5 9 14 13 ...
 $ marital.status: Factor w/ 7 levels " Divorced"," Married-AF-spouse",..: 5 3 1 3 3 3 4 3 5 3 ...
 $ occupation    : Factor w/ 14 levels " Adm-clerical",..: 1 4 6 6 10 4 8 4 10 4 ...
 $ relationship  : Factor w/ 6 levels "0","1","2","3",..: 1 2 1 2 3 3 1 2 1 2 ...
 $ race          : Factor w/ 5 levels "0","1","2","3",..: 1 1 1 2 2 1 2 1 1 1 ...
 $ sex           : Factor w/ 2 levels "0","1": 2 2 2 2 1 1 1 2 1 2 ...
 $ capital.gain  : int  2174 0 0 0 0 0 0 0 14084 5178 ...
 $ capital.loos  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ hour.per.week : int  40 13 40 40 40 40 16 45 50 40

      age                    workclass      final.weight    
 Min.   :17.00    Federal-gov     :  943   Min.   :  13769  
 1st Qu.:28.00    Local-gov       : 2067   1st Qu.: 117627  
 Median :37.00    Private         :22286   Median : 178425  
 Mean   :38.44    Self-emp-inc    : 1074   Mean   : 189794  
 3rd Qu.:47.00    Self-emp-not-inc: 2499   3rd Qu.: 237628  
 Max.   :90.00    State-gov       : 1279   Max.   :1484705  
                  Without-pay     :   14                    
         education    education.num                  marital.status 
  HS-grad     :9840   Min.   : 1.00    Divorced             : 4214  
  Some-college:6678   1st Qu.: 9.00    Married-AF-spouse    :   21  
  Bachelors   :5044   Median :10.00    Married-civ-spouse   :14065  
  Masters     :1627   Mean   :10.12    Married-spouse-absent:  370  
  Assoc-voc   :1307   3rd Qu.:13.00    Never-married        : 9726  
  11th        :1048   Max.   :16.00    Separated            :  939  
 (Other)      :4618          

,age,workclass,final.weight,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loos,hour.per.week,native.country,income
,<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,0,0,1,2174,0,40,0,0
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,1,0,1,0,0,13,0,0
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,0,0,1,0,0,40,0,0
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,1,1,1,0,0,40,0,0
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,2,1,0,0,0,40,1,0
6,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,2,0,0,0,0,40,0,0


In [65]:
## 3) Normalizando os dados

data[,1] = scale(data[,1])
data[,3] = scale(data[,3])
data[,5] = scale(data[,5])
data[,11:13] = scale(data[,11:13])
head(data)

,age,workclass,final.weight,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loos,hour.per.week,native.country,income
,"<dbl[,1]>",<fct>,"<dbl[,1]>",<fct>,"<dbl[,1]>",<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
1,0.04279500,5,-1.0627040,9,1.1288997,4,0,0,0,1,0.1460899,-0.2185824,-0.07773282,0,0
2,0.88027355,4,-1.0078546,9,1.1288997,2,3,1,0,1,-0.1474422,-0.2185824,-2.33149205,0,0
3,-0.03333941,2,0.2446894,11,-0.4397309,0,5,0,0,1,-0.1474422,-0.2185824,-0.07773282,0,0
4,1.10867679,2,0.4252333,1,-1.2240462,2,5,1,1,1,-0.1474422,-0.2185824,-0.07773282,0,0
5,-0.79468355,2,1.4066350,9,1.1288997,2,9,2,1,0,-0.1474422,-0.2185824,-0.07773282,1,0
6,-0.10947383,2,0.8971652,12,1.5210573,2,3,2,0,0,-0.1474422,-0.2185824,-0.07773282,0,0


In [29]:
#Instalando pacote para preparação dos dados de ML

install.packages("caTools")
library(caTools)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’




In [66]:
#4) Separando as Bases de dados em treino e Teste

set.seed(1) #Determinando o número de sementes do algorítmo

divisao = sample.split(data$income, SplitRatio = 0.85) #dividindo o dataset em 75% para treinamento e 25% teste, o retorno será um vetor TRUE or False que determinará os registros que farão parte do dataset.

data_trein = subset(data, divisao == TRUE) #Definindo o dataset de treinamento
data_tst = subset(data, divisao == FALSE) #Definindo o dataset de teste

nrow(data_trein)
nrow(data_tst)

[1] 25638

[1] 4524

In [32]:
# Instalando o Pacote de ML (Que contém o algorítmo SVM)

install.packages("e1071")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [0]:
library(e1071)

In [0]:
# 5) Criando o algorítmo de Classificação


class = svm(formula = income ~., data = data_trein, type = "C-classification", kernel = "radial")



In [0]:
# 6) Prevendo as condições futuras dos dados de treino

prev = predict(class, newdata = data_tst[-15])

In [51]:
mtx_conf = table(data_tst[,15],prev)

mtx_conf

   prev
       0    1
  0 3180  218
  1  498  628

In [53]:
install.packages("caret")
library(caret)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘data.table’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘recipes’, ‘pROC’


Loading required package: lattice



In [54]:
confusionMatrix(mtx_conf)

Confusion Matrix and Statistics

   prev
       0    1
  0 3180  218
  1  498  628
                                          
               Accuracy : 0.8417          
                 95% CI : (0.8308, 0.8523)
    No Information Rate : 0.813           
    P-Value [Acc > NIR] : 2.348e-07       
                                          
                  Kappa : 0.5383          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.8646          
            Specificity : 0.7423          
         Pos Pred Value : 0.9358          
         Neg Pred Value : 0.5577          
             Prevalence : 0.8130          
         Detection Rate : 0.7029          
   Detection Prevalence : 0.7511          
      Balanced Accuracy : 0.8035          
                                          
       'Positive' Class : 0               
                                          